In [ ]:
!pip install datasets evaluate transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
#Импорт библиотек
import pandas as pd
import random
import torch
import numpy as np

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

from datasets import load_metric
from datasets import load_dataset

In [ ]:
#Загружаем датасет и ограничиваем его 2048 символами
df = pd.read_csv('news.csv', nrows = 4000)
df['text'] = df['text'].str[:2048]
len(df.iloc[1]['text'])

2048

In [ ]:
#Загружаем датасет в новый файл df_news.csv
df.to_csv('news_333.csv', index = False)
df

,text,label
0,Помощник депутата Палаты представителей: мы да...,1
1,"Вы когда-нибудь чувствовали, что ваша жизнь дв...",0
2,Почему правда может привести к увольнению 29 о...,1
3,"Выявлено 15 мирных жителей, погибших в результ...",1
4,Распечатать\nИранская женщина была приговорена...,1
...,...,...
3995,Большая фармацевтика нацелена на прибыль от ре...,1
3996,Главная / #Решения / Кризис совести? Обама осв...,1
3997,ВАШИНГТОН. На днях сенатор от штата Айова Чарл...,0
3998,Натан Глейзер и Дэниел Патрик Мойнихан\nЗа пре...,1


In [ ]:
from sklearn.model_selection import train_test_split
df_news = load_dataset('csv', data_files = '/content/news_333.csv', split = 'train') #загружаем датасет news.csv(из интернета)
df_news

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-63b1304084d5f6bb/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 4000
})

In [ ]:
df_news = df_news.train_test_split(test_size = 0.3) #разделение на трейн и тест выборку
df_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [ ]:
train = df_news['train'] #трейн выборка из 10500 строк
test = df_news['test']

In [ ]:
#посмотрим на данные
print('text: ', train[3]['text'])
print('is_fake: ', train[3]['label'])

text:  Эл. адрес
Кандидат в президенты от Демократической партии Хиллари Клинтон отреагировала на последнюю громкую новость о своих электронных письмах, потребовав от директора ФБР Джеймса Коми немедленно обнародовать все, что обнаружило ФБР.
Она также ложно обвинила Коми в том, что он отправляет письма только республиканцам, предположив, что тем самым он вызвал политическую бурю. Клинтон провела короткую четырехминутную пресс-конференцию с журналистами в Айове в пятницу, где она проводила предвыборную кампанию.
«Теперь я ознакомилась с письмом директора Коми Конгрессу, — сказала Клинтон, открывая свое выступление. «Нам осталось 11 дней до, возможно, самых важных общенациональных выборов в нашей жизни… поэтому американский народ заслуживает того, чтобы немедленно получить полные и исчерпывающие факты». https://twitter.com/NBCNightlyNews/status/792142514471907329
«Сам директор сказал, что не знает, являются ли электронные письма, упомянутые в его письме, важными или нет», — продолжила о

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels = 2)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2

In [ ]:
def tokenize_function(data):
  return tokenizer(data['text'])

tokenized_train = train.map(tokenize_function)
tokenized_test = test.map(tokenize_function)

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.remove_columns('text')
tokenized_test = tokenized_test.remove_columns('text')

In [ ]:
tokenized_train

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2800
})

In [ ]:
metric = load_metric('f1')
def compute_metrics(eval):
  logits, labels = eval
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

<ipython-input-13-924dc9bbaf81>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1')


In [ ]:
from huggingface_hub import notebook_login
notebook_login() #Hugging Face token

In [ ]:
training_args = TrainingArguments(
    "lfewfwe",
    num_train_epochs = 4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 16,
    learning_rate = 1e-5,
    evaluation_strategy = 'epoch',
    logging_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit = 1,
    seed = 21,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_train,
    compute_metrics = compute_metrics
)
trained = trainer.train()

Cloning https://huggingface.co/YuryCHep/lfewfwe into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.555100,0.314567,0.856661
2,0.277400,0.225782,0.907620


Epoch,Training Loss,Validation Loss,F1
1,0.555100,0.314567,0.856661
2,0.277400,0.225782,0.907620
3,0.204300,0.170882,0.942244
4,0.167800,0.158322,0.947325


In [ ]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis", model = 'YuryCHep/lfewfwe')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
df

,text,label
0,Помощник депутата Палаты представителей: мы да...,1
1,"Вы когда-нибудь чувствовали, что ваша жизнь дв...",0
2,Почему правда может привести к увольнению 29 о...,1
3,"Выявлено 15 мирных жителей, погибших в результ...",1
4,Распечатать\nИранская женщина была приговорена...,1
...,...,...
3995,Большая фармацевтика нацелена на прибыль от ре...,1
3996,Главная / #Решения / Кризис совести? Обама осв...,1
3997,ВАШИНГТОН. На днях сенатор от штата Айова Чарл...,0
3998,Натан Глейзер и Дэниел Патрик Мойнихан\nЗа пре...,1


In [ ]:

print(nlp('Пригожин взломан, аудиосообщение сгенерированы ИИ – источник. Каналы Пригожина взломали, аудиосообщения от его имени сгенерированы нейросетью, сообщили сразу два независимых источника, связанных с ЧВК "Вагнер. Уже из двух независимых источников мне написали, что "Вагнера" просят передать, что происходящее — вброс, сообщает ополченец с позывным Топаз. По его словам, каналы Евгения Пригожина в Telegram были взломаны, а аудиосообщения от его имени - сгенерированы нейросетью. Это косвенно подтверждает резкое изменение формата сообщений основателя ЧВК "Вагнер". Ранее все комментарии Пригожина были записаны в виде голосовых сообщений. Однако последние аудиосообщения – это файлы в формате mp3.'))

[{'label': 'LABEL_1', 'score': 0.9886660575866699}]


In [ ]:
print(nlp('Праздник выпускников «Алые паруса» начался на Дворцовой площади в Санкт-Петербурге. Об этом сообщает РБК.Мероприятие открыл губернатор Петербурга Александр Беглов. Отмечается, что движение на Невском проспекте перекрыто, а Росгвардия пропускает только молодых людей. На улицах города дежурят полиция, ДПС, ОМОН и пожарные.Для выпускников на сцене выступят группа «Чайф», Юлианна Караулова, Полина Гагарина, Gafur, Моя Мишель, Burito, Тося Чайкина, The Hatters, Gayazovs Brothers и белорусский певец LYRIQ.Ранее официальный представитель Кремля Дмитрий Песков сообщил, что президент России Владимир Путин не планирует посещать праздник, отвечая на вопрос, есть ли в графике главы государства такое мероприятие.'))

[{'label': 'LABEL_0', 'score': 0.9867768883705139}]


In [ ]:
print(nlp('''Кремль сообщил, что бойцов ЧВК «Вагнер» не будут преследовать
Песков: не подписавших контракт с Минобороны бойцов ЧВК «Вагнер» не будут преследовать
Ситуацию с Пригожиным урегулировали «без дальнейших потерь», а бойцов, участвовавших в планах бизнесмена, не будут преследовать, пообещали в Кремле. Сам Пригожин отправится в Белоруссию по договоренности с Лукашенко

Фото: Эрик Романенко / ТАСС
Бойцы ЧВК «Вагнер», изначально отказавшиеся от участия в объявленных Пригожиным планах «идти на Москву», смогут подписать контракты с Минобороны, против остальных не будут заведены уголовные дела с учетом их заслуг, заявил представитель президента Дмитрий Песков, передает корреспондент РБК.

«Их никто не будет преследовать с учетом их заслуг на фронте. Мы всегда с уважением относились к их героическим поступкам на фронте. И будем всегда это уважение хранить», — пообещал Песков.

Он отметил, что ситуацию удалось разрешить «без дальнейших потерь» и роста эскалации. Представитель президента добавил также, что некоторые подразделения ЧВК «с самого начала одумались и вернулись», запросив сопровождение ГАИ.

Дело против основателя ЧВК «Вагнер» Евгения Пригожина прекратят, бизнесмен уедет в Белоруссию, рассказал также Песков.'''))

[{'label': 'LABEL_0', 'score': 0.5385636687278748}]
